<a href="https://colab.research.google.com/github/etalab-ia/ami-ia/blob/master/notebooks/intro-ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook NLP

Introduction au NLP avec l'examples de la consultation 

In [0]:
# import libraries
import pandas as pd
import numpy as np

In [0]:
# importer un fichier depuis votre ordinateur local 
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving completez-le-questionnaire-en-quelques-minutes.csv to completez-le-questionnaire-en-quelques-minutes.csv
User uploaded file "completez-le-questionnaire-en-quelques-minutes.csv" with length 152951273 bytes


In [0]:
# lire le jeu de données en un objet "pandas dataframe"
df = pd.read_csv("completez-le-questionnaire-en-quelques-minutes.csv", sep = ",", header = 0)

In [0]:
# on renomme les colonnes des questions et on conserve un dictionnaire avec la correspondance 
colnames = list(df.columns)
dict_questions = dict(zip(["question_" + str(x) for x in range(1, 17)], colnames[6:23]))
colnames_new = colnames[0:6] + ["question_" + str(x) for x in range(1, 17)] + ["age"]
df.columns = colnames_new

In [0]:
dict_questions

{'question_1': 'QUXVlc3Rpb246MTI5 - La mise en place d’un système universel de retraites par répartition répond à plusieurs objectifs. Parmi les objectifs ci-dessous, lesquels vous semblent prioritaires ?',
 'question_10': 'QUXVlc3Rpb246MTQ3 - Dans le futur système universel de retraites, il y aura comme aujourd’hui un âge minimal de départ et un âge permettant de bénéficier de 100% de sa retraite (taux plein). Parmi les propositions suivantes, lesquelles vous paraissent prioritaires :',
 'question_11': 'QUXVlc3Rpb246MTQ5 - Dans le futur système universel de retraites, des points seront accordés aux parents qui ont eu des enfants afin de compenser l’impact que cela a eu sur leur carrière professionnelle. Il est prioritaire selon vous de :',
 'question_12': 'QUXVlc3Rpb246MTUx - Dans le futur système universel de retraites, il y aura un dispositif de réversion afin que les veuves et les veufs touchent une partie de la retraite de leur conjoint décédé. Parmi les propositions suivantes, le

In [0]:
df.head()

,id,createdAt,publishedAt,updatedAt,authorId,authorType,question_1,question_2,question_3,question_4,question_5,question_6,question_7,question_8,question_9,question_10,question_11,question_12,question_13,question_14,question_15,question_16,age
0,UmVwbHk6MDAwMDZkNjctMThmMi0xMWVhLWE0ODUtZmExNj...,2019-12-07 14:03:45,2019-12-07 14:03:45,2019-12-07 14:03:45,VXNlcjpjY2RkNzU3Mi0xOGYwLTExZWEtYTQ4NS1mYTE2M2...,NaN,Il n’est pas nécessaire de mettre en place un ...,La mise en place d'un système en points n'est ...,NaN,Les inégalités de pensions entre les femmes et...,Garantir une retraite minimum aux personnes qu...,Attribuer autant de points que si la personne ...,Attribuer autant de points que si la personne ...,"Fixer le même minimum de retraite pour tous, q...",Autoriser le départ avant 62 ans pour les pers...,Conserver un âge minimum de départ à 62 ans|Ga...,Accorder des points en priorité aux femmes|Aug...,Les règles de réversion doivent être les mêmes...,Faire payer un peu plus de cotisations sociale...,Faire cotiser les salariés du public sur l’int...,"Permettre aux personnes qui le souhaitent, dès...",Les partenaires sociaux (syndicats de salariés...,65 - 69 ans
1,UmVwbHk6MDAwMGY0N2QtMTE0OS0xMWVhLWE0ODUtZmExNj...,2019-11-27 20:06:22,2019-11-27 20:06:22,2019-11-27 20:06:22,VXNlcjpiMjY4YTNkMi0xMTQ2LTExZWEtYTQ4NS1mYTE2M2...,NaN,Il n’est pas nécessaire de mettre en place un ...,La mise en place d'un système en points n'est ...,Uniquement les personnes éloignées du départ à...,Les inégalités de pension entre les retraités ...,Donner des points pour les périodes de maladie...,Attribuer autant de points que si la personne ...,Attribuer autant de points que si la personne ...,Garantir le montant du minimum de retraite en ...,Autoriser le départ avant 62 ans des personnes...,Garantir 100% de la retraite (retraite à taux ...,Augmenter de 5% (à partager entre les parents)...,Accorder la pension de réversion aux couples m...,Faire payer un peu plus de cotisations sociales,Faire cotiser les salariés du public sur l’int...,Il n’y a pas lieu d’encourager l’emploi des pe...,Une combinaison de tous ces acteurs,60 - 64 ans
2,UmVwbHk6MDAwMWUzYTctMTVjYy0xMWVhLWE0ODUtZmExNj...,2019-12-03 13:54:11,2019-12-03 13:54:27,2019-12-03 13:54:27,VXNlcjozODgwZjgzOS0xNWNhLTExZWEtYTQ4NS1mYTE2M2...,Citoyen,Avoir un seul système de retraite pour tous le...,Que chaque jour travaillé et rémunéré permette...,Uniquement les personnes éloignées du départ à...,Les inégalités de pension entre les retraités ...,Garantir une retraite minimum aux personnes qu...,Attribuer autant de points que si la personne ...,Attribuer des points à hauteur de l'allocation...,"Fixer le même minimum de retraite pour tous, q...",Autoriser le départ avant 62 ans des personnes...,Conserver une décote (réduction du montant de ...,Accorder des points en priorité aux femmes|Acc...,Les règles de réversion doivent être les mêmes...,Baisser un peu le montant des pensions versées...,"Avoir le même taux de cotisation pour tous, qu...",Faciliter le passage à temps partiel en fin de...,Une combinaison de tous ces acteurs,50 - 54 ans
3,UmVwbHk6MDAwMzA3NWEtMThkNC0xMWVhLWE0ODUtZmExNj...,2019-12-07 10:29:00,2019-12-07 10:43:52,2019-12-07 10:43:53,VXNlcjplNTZmNWRiZi0xOGNmLTExZWEtYTQ4NS1mYTE2M2...,Citoyen,Garantir l’équilibre financier du système de r...,Garantir qu'un euro cotisé donne le même nombr...,"Tout le monde, sauf les retraités et les gens ...",Les inégalités sur l’âge de départ à la retrai...,Donner des points aux personnes qui ont dû s’a...,"Attribuer des points, mais moins que si la per...",Attribuer des points à hauteur de l'allocation...,régime unique fonction publique - privé (les r...,Autoriser le départ avant 62 ans des personnes...,Conserver une décote (réduction du montant de ...,"Accorder les mêmes droits, que vous soyez sala...",Les règles de réversion doivent être les mêmes...,Faire travailler les Français un peu plus long...,Faire cotiser les salariés du public sur l’int...,Permettre a